In [19]:
import pandas as pd # Data Analysis library for working with tabular data

# Load our AtlasIV data from the CSV file
df = pd.read_csv('AtlasIV.csv')
df.head()

,day,regen,battery,reflector,distance,data,radiation
0,1,N,89.4,5.0,1296,241,10.3
1,2,N,88.6,NaN,1291,254,10.2
2,3,Y,91.9,1.0,1334,169,10.2
3,4,N,89.9,NaN,1301,214,10.3
4,5,Y,90.5,4.0,1203,204,10.3


In [20]:
# Clean the regen column to be 0 for no recharge or 1 for recharged
df['regen'].where(~(df['regen'] == 'N'), other=0, inplace=True)
df['regen'].where(~(df['regen'] == 'Y'), other=1, inplace=True)
df['regen'] = df['regen'].astype(int)

df.head()

,day,regen,battery,reflector,distance,data,radiation
0,1,0,89.4,5.0,1296,241,10.3
1,2,0,88.6,NaN,1291,254,10.2
2,3,1,91.9,1.0,1334,169,10.2
3,4,0,89.9,NaN,1301,214,10.3
4,5,1,90.5,4.0,1203,204,10.3


In [21]:
df.describe()

,day,regen,battery,reflector,distance,data,radiation
count,365.000000,365.000000,365.000000,196.000000,365.000000,365.000000,365.000000
mean,183.000000,0.534247,85.959178,2.923469,1227.616438,315.205479,10.238904
std,105.510663,0.499511,2.706698,1.463841,59.161037,74.287413,0.225000
min,1.000000,0.000000,78.400000,1.000000,1061.000000,105.000000,9.700000
25%,92.000000,0.000000,84.000000,1.750000,1189.000000,262.000000,10.100000
50%,183.000000,1.000000,86.000000,3.000000,1228.000000,317.000000,10.200000
75%,274.000000,1.000000,88.000000,4.000000,1268.000000,367.000000,10.300000
max,365.000000,1.000000,92.700000,5.000000,1377.000000,497.000000,11.600000
